In [43]:
# Multiple horizons MTM analysis
include("/Users/federicoclaudi/Documents/Github/LocomotionControl/analysis/behavior/analysis_fixtures.jl")
include("/Users/federicoclaudi/Documents/Github/LocomotionControl/analysis/behavior/analysis_visuals.jl")

import jcontrol as jc
using Printf
import jcontrol.control: default_control_options
import jcontrol.bicycle: Bicycle
import IterTools: product as ×
import jcontrol: Track, toDict
import InfiniteOpt: termination_status

import jcontrol.Run: run_mtm

  Activating project at `~/Documents/Github/LocomotionControl/analysis/behavior`


@Info (Main):  Loaded 1021 trials 
  ╰──────────────────────────────────────────────── 
                      Tue, 24 May 2022 10:39:17 
@Info (Main):  After tortuosity analysis, discarded 9.109% of trials | 928

 
  │                trials left 
  ╰──────────────────────────────────────────────── 
                      Tue, 24 May 2022 10:39:18 


# MTM on track
Solve the MTM prolblem on a bunch of tracks and with slightly different models paramters to created data to use for training RNNs.

## Create bikes with different parameters

In [24]:
default_bike = Bicycle()
variables = (:l_f, :l_r, :width, :c)

σ = .05
n = 3

defaults = Dict(v=>getfield(default_bike, v) for v in variables)
variables_values = Dict(
    v => collect(range(defaults[v]-σ*defaults[v], stop=defaults[v]+σ*defaults[v], length=n)) for v in variables
)

n_bikes = length(collect(×(values(variables_values)...)))

81

## Load tracks

In [37]:
fld = "/Users/federicoclaudi/Documents/Github/LocomotionControl/analysis/RNN/tracks"
tracks_files = glob("*.npy", fld)
n_tracks = length(tracks_files)

16

In [25]:
N = length(
    ×(
        values(variables_values)...,
        tracks_files
    )
)

print(
    "Tot number of simulations = $N"
)

Tot number of simulations = 1296

## Simulate all

In [44]:
save_fld = "/Users/federicoclaudi/Dropbox (UCL)/Rotation_vte/Locomotion/analysis/RNN/tracks_mtm_sims"

i = 0
for lf in variables_values[:l_f], lr in variables_values[:l_r], w in variables_values[:width], c in variables_values[:c]
    bike = Bicycle(
        l_f=lf,
        l_r=lr,
        width=w,
        c=c
    )
    i += 1  
    @info "Doing bike $i/$n_bikes"


    for trackfile in tracks_files
        track_name = splitext(splitpath(trackfile)[end])[1]

        savepath = joinpath(save_fld, "$(track_name)_bike_$i.csv")
        isfile(savepath) && return

        track = Track(; start_waypoint=4, npyfile=trackfile, const_width=true, track_length=nothing)

        _, _, control_model, solution = run_mtm(
            :dynamics,  # model type
            2.0;  # supports density
            showtrials=nothing,
            showplots=false,
            quiet=true,
            bike=bike,
            track=track,
        )
        if "LOCALLY_SOLVED" == string(termination_status(control_model))
            data = DataFrame(toDict(solution))
            CSV.write(savepath, data)
        else
            @warn "FAILED" termination_status(control_model) track_name bike
        end
        
    end
end


@Info (Main.top-level scope):  Doing bike 1/81 
  ╰──────────────────────────────────────────────── 
                      Tue, 24 May 2022 10:39:21 
@Warn (Main.top-level scope):  FAILED

 
  │ 
  │ (MathOptInterface.TerminationStatusCode)  ▶  termination_status(control_model) = LOCALLY_INFEASIBLE 
  │ (String)                                  ▶  track_name                        = cosine_chirp 
  │ (Bicycle)                                 ▶  bike                              = Bicycle(2.85, 1.9, 4.75, 1.9, 23, 137.18, 3800.0, 0.8, 16, "#37474F", "#00ACC1", 8, "#78909C")

 
  ╰──────────────────────────────────────────────── 
                      Tue, 24 May 2022 10:41:31 


@Warn (Main.top-level scope):  FAILED 
  │ 
  │ (MathOptInterface.TerminationStatusCode)  ▶  termination_status(control_model) = LOCALLY_INFEASIBLE 
  │ (String)                                  ▶  track_name                        = sine_chirp 
  │ (Bicycle)                                 ▶  bike                              = Bicycle(2.85, 1.9, 4.75, 1.9, 23, 137.18, 3800.0, 0.8, 16, "#37474F", "#00ACC1", 8, "#78909C") 
  ╰──────────────────────────────────────────────── 
                      Tue, 24 May 2022 10:44:21 
@Info (Main.top-level scope):  Doing bike 2/81

 
  ╰──────────────────────────────────────────────── 
                      Tue, 24 May 2022 10:46:38 


...

@Warn (Main.top-level scope):  FAILED 
  │ 
  │ (MathOptInterface.TerminationStatusCode)  ▶  termination_status(control_model) = LOCALLY_INFEASIBLE 
  │ (String)                                  ▶  track_name                        = sine_chirp 
  │ (Bicycle)                                 ▶  bike                              = Bicycle(2.85, 2.0, 4.85, 2.1, 23, 141.47, 3800.0, 0.8, 16, "#37474F", "#00ACC1", 8, "#78909C") 
  ╰──────────────────────────────────────────────── 
                      Tue, 24 May 2022 12:24:08 
@Info (Main.top-level scope):  Doing bike 17/81

 
  ╰──────────────────────────────────────────────── 
                      Tue, 24 May 2022 12:25:53 


@Warn (Main.top-level scope):  FAILED 
  │ 
  │ (MathOptInterface.TerminationStatusCode)  ▶  termination_status(control_model) = LOCALLY_INFEASIBLE 
  │ (String)                                  ▶  track_name                        = cosine_chirp 
  │ (Bicycle)                                 ▶  bike                              = Bicycle(2.85, 2.0, 4.85, 2.1, 23, 141.47, 4000.0, 0.8, 16, "#37474F", "#00ACC1", 8, "#78909C") 
  ╰──────────────────────────────────────────────── 
                      Tue, 24 May 2022 12:27:28 
@Warn (Main.top-level scope):  FAILED

 
  │ 
  │ (MathOptInterface.TerminationStatusCode)  ▶  termination_status(control_model) = LOCALLY_INFEASIBLE 
  │ (String)                                  ▶  track_name                        = sine_chirp 
  │ (Bicycle)                                 ▶  bike                              = Bicycle(2.85, 2.0, 4.85, 2.1, 23, 141.47, 4000.0, 0.8, 16, "#37474F", "#00ACC1", 8, "#78909C") 
  ╰──────────────────────────────────────────────── 
                      Tue, 24 May 2022 12:30:08 


@Info (Main.top-level scope):  Doing bike 18/81 
  ╰──────────────────────────────────────────────── 
                      Tue, 24 May 2022 12:31:59 
@Warn (Main.top-level scope):  FAILED